In [123]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from numpy.linalg import lstsq
from itertools import combinations
from statsmodels.api import OLS
from sklearn.linear_model import LassoCV, RidgeCV
from tqdm import tqdm
import statsmodels.api as sm
from scipy import stats

In [86]:
## Fetching data - using value weighted returns

industries = pd.read_csv("data/17_Industry_Portfolios.csv",index_col="Date")
industries.index = pd.to_datetime(industries.index,format="%Y%m")
industries = industries.resample("M").last()

mom_size = pd.read_csv("data/MOM-size.csv",index_col="Date")
mom_size.index = pd.to_datetime(mom_size.index,format="%Y%m")
mom_size = mom_size.resample("M").last()

size_value = pd.read_csv("data/size_value.csv",index_col="Date")
size_value.index = pd.to_datetime(size_value.index,format="%Y%m")
size_value = size_value.resample("M").last()

# momentum
momentum = pdr.get_data_famafrench("F-F_Momentum_Factor_daily", start="1920", end="2020-12-31")[0]
momentum = momentum.resample("M").last().squeeze()
momentum.rename(level=0,index="Mom")

momentum = pd.read_csv("data/F-F_Momentum_Factor.csv",index_col="Date")
momentum.index = pd.to_datetime(momentum.index,format="%Y%m")
momentum = momentum.resample("M").last().squeeze()
# Value weighted market
vwm = pd.read_csv("data/VWM.csv",index_col="Date")
vwm.index = pd.to_datetime(vwm.index, format="%Y%m")
vwm = vwm.resample("M").last()

In [122]:

industries = industries["1927-01-31":]
momentum = momentum["1927-01-31":]
vwm = vwm["1927-01-31":]
hml = 0.5 * (size_value["SMALL HiBM"] + size_value["BIG HiBM"]) - 0.5 * (size_value["SMALL LoBM"] + size_value["BIG LoBM"] )
hml = hml["1927-01-31":]
industries

,Food,Mines,Oil,Clths,Durbl,Chems,Cnsum,Cnstr,Steel,FabPr,Machn,Cars,Trans,Utils,Rtail,Finan,Other
Date,,,,,,,,,,,,,,,,,
1927-01-31,-0.82,1.00,1.71,-1.18,4.66,0.40,0.34,-2.78,-0.39,-2.29,-0.58,-1.09,1.24,-1.79,-2.53,-2.38,1.77
1927-02-28,4.56,5.38,0.98,4.15,2.29,9.77,1.60,4.57,3.67,6.11,5.34,9.55,5.13,4.53,3.61,2.97,4.32
1927-03-31,1.81,4.86,-7.00,-0.93,-2.48,6.00,4.86,3.91,1.48,-9.00,0.66,1.75,0.79,0.37,-0.39,1.36,3.62
1927-04-30,2.73,2.14,-5.81,4.42,2.31,3.09,3.76,-0.71,-0.34,5.14,4.98,3.11,0.60,1.71,4.47,2.89,-1.90
1927-05-31,6.21,1.49,4.97,6.00,8.34,4.00,10.85,5.90,3.22,6.38,7.73,5.91,7.30,9.21,2.34,10.25,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31,5.81,9.26,-4.98,-0.29,12.32,6.84,4.44,9.20,2.34,3.06,6.03,18.10,2.41,6.37,10.08,1.96,6.46
2020-08-31,4.08,4.94,-1.27,12.21,6.05,4.73,3.54,7.78,8.18,8.83,5.85,38.71,10.40,-2.25,8.66,4.97,9.58
2020-09-30,-1.83,-1.09,-15.37,6.28,3.73,0.29,-2.52,-0.05,-4.90,-0.17,-0.90,-9.18,-0.85,-0.27,-4.42,-4.08,-4.52


In [100]:
hml = 0.5 * (size_value["SMALL HiBM"] + size_value["BIG HiBM"]) - 0.5 * (size_value["SMALL LoBM"] + size_value["BIG LoBM"] )

In [88]:
## Cross Validation function

def xval_5fold(y, x , random=False, seed = 20201231,fold=5):
    # Use numpy arrays for simplicity
    y = np.asarray(y)
    x = np.asarray(x)
    n = y.shape[0]
    if random:
        # If randomisation is needed, use either the default or provided seed
        rg = np.random.default_rng(seed)
        # Generate a set of index values to use to randomly reorder the data
        # After randomisation, we can use the data as if it is is inorder!
        ind = rg.permutation(np.arange(n))
        y = y[ind]
        x = x[ind]
    # Compute the block size
    block = n / fold
    sse = 0.0
    for i in range(int(fold)):
        # Start and end of each block need to be integers or we lose an observation
        # Rounding ensures that we get all observations since int rounds down
        st = int(np.round(i * block))
        en = int(np.round((i + 1) * block))
        # Construct the indicies of the observations that we leave out
        leave_out = np.r_[st:en]
        # The included are the one that we don't leave out
        include = np.setdiff1d(np.arange(n), leave_out)
        # Compute the regression coefficients
        beta = lstsq(x[include], y[include], rcond=None)[0]
        # Compute the residuals and add to the sse
        resid = y[st:en]- x[st:en] @ beta
        sse += resid @ resid
    return sse

# Randomisation is better when we suspect the model may not be totally stable

In [91]:
# forward stepwise function: 
# requires x: x dataframe , y series, p = number of columns, requires xval function too
# input x, y, p
def forward_stepwise(x, y, p,random=False,fold=5):
    included = []

    for i in range(p):
        excluded = [col for col in x if col not in included]
        best_sse = np.inf
        for col in excluded:
            try_x = x[included + [col]]
            beta = lstsq(try_x, y, rcond=None)[0]
            resid = y - try_x @ beta
            sse = resid @ resid
            if sse < best_sse:
                best_sse = sse
                next_var = col
                included.append(next_var)
    
    fsr_sse_sv = {}
    for i in range(1,p+1):
        fsr_sse_sv[i] = xval_5fold(y,x[included[:i]],random=random, fold=fold)
    fsr_sse_sv = pd.Series(fsr_sse_sv)
    forward_step_model = included[:fsr_sse_sv.idxmin()]
    return forward_step_model

In [117]:
### LASSO function

def lasso_reg(x,y):

    x_scale = x.std(ddof=0)
    y_scale = y.std(ddof=0)
    std_x = x / x_scale
    std_y = y / y_scale
    std_x = std_x

    lasso_cv = LassoCV(fit_intercept=False)
    lasso_cv = lasso_cv.fit(std_x,std_y)
    #print(f"Optimal Alpha = {lasso_cv.alpha_}")
    lasso_beta = lasso_cv.coef_  * (y_scale / x_scale)
    return lasso_beta


In [124]:
# General to Specific selection

def gts(x,y):
    cv = stats.norm.ppf(.995)

    included  = list(x.columns)
    y = y
    for i in range(16):
        x = x.loc[:,included]
        res = sm.OLS(y,x).fit(cov_type="HC0")
        tstats = res.tvalues
        if tstats.abs().min() < cv:
            sorted_tstats = tstats.abs().sort_values()
            remove = sorted_tstats.index[0]
            included.remove(remove)
        else:
            break
    return included

In [142]:
# Ridge regression 

def ridge_reg(x,y):
    x_scale = x.std(ddof=0)
    y_scale = y.std(ddof=0)
    std_x = x / x_scale
    std_y = y / y_scale

    ridge_cv = RidgeCV(fit_intercept=False, alphas=np.linspace(1, 100, 100))
    ridge_cv = ridge_cv.fit(std_x, std_y)
    #print(f"Optimal alpha = {ridge_cv.alpha_}")
    start = ridge_cv.alpha_ - 1 
    end = ridge_cv.alpha_ + 1

    ridge_cv = RidgeCV(fit_intercept=False, alphas=np.linspace(start, end, 2001))
    ridge_cv = ridge_cv.fit(std_x, std_y)
    #print(f"Optimal alpha = {ridge_cv.alpha_}")
    return ridge_cv.coef_ * (y_scale / x_scale)

ridge_reg(industries,hml)

Food     0.038112
Mines   -0.038744
Oil      0.068052
Clths    0.023243
Durbl    0.145572
Chems   -0.075729
Cnsum   -0.132241
Cnstr   -0.048827
Steel    0.183236
FabPr   -0.141720
Machn   -0.282974
Cars     0.072488
Trans    0.292431
Utils    0.123949
Rtail   -0.140235
Finan    0.268155
Other   -0.359207
dtype: float64

HML (Value), 20 year rolling
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [143]:

nobs = hml.shape[0]

# 20 year rolling regressions
for i in range(240,nobs-60,12): 
    # Select the in sample training data
    y = hml.iloc[i-240:i]
    y = y.squeeze()
    x = industries.iloc[i-240:i]
    t, p = x.shape
    
    # Form the out of sample data
    y_oos = (hml.iloc[i:i+60]).squeeze()
    x_oos = industries.iloc[i:i+60]

    #---------------------------------------------------------------------------------------------------------
    # Simple OLS Regression
    res = sm.OLS(y,x).fit(cov_type="HC0")
    pred_ols = x_oos @ res.params
    resid_oos = y_oos - pred_ols
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("Simple OLS")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Forward Stepwise
    model = forward_stepwise(x, y, p)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    
    pred_fwd = x_oos[model] @ res.params
    resid_oos = y_oos - pred_fwd
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on the forward stepwise
    beta = lasso_reg(x[model],y)
    pred_fwd_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_fwd_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("Lasso on Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso regression
    beta = lasso_reg(x,y)
    pred_lasso = x_oos @ beta
    resid_oos = y_oos - pred_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("Lasso")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

     #---------------------------------------------------------------------------------------------------------
    # Ridge selection
    beta = ridge_reg(x,y)
    pred_ridge = x_oos @ beta
    resid_oos = y_oos - pred_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("Ridge")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # General to Specific selection
    model = gts(x,y)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    pred_gts = x_oos[model] @ res.params
    resid_oos = y_oos - pred_gts
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on general to specific
    beta = lasso_reg(x[model],y)
    pred_gts_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    #print("Lasso on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Mixed model
    pred_mixed = (1/3)* pred_fwd + (1/3) * pred_fwd_lasso + (1/3)*pred_lasso
    resid_oos = y_oos - pred_mixed
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss

    """
    print("Naive Averaging")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """





Ridge
104.02028596274417
0.7730531090341923

Ridge
107.65824156932366
0.7575999064739651

Ridge
121.74339659178858
0.6980392192451128

Ridge
134.94452255750517
0.7166583399833731

Ridge
136.12612306077213
0.4135307394393579

Ridge
149.7851237522538
0.13747953335933583

Ridge
135.5559491869708
0.2937796910554684

Ridge
175.5235980549565
0.13695781631197101

Ridge
207.18922071514655
-0.44166409666583006

Ridge
181.92264839020942
-0.0005933205281634102

Ridge
187.69674933743914
0.01293610684923352

Ridge
163.60060039985052
0.25569358470747283

Ridge
147.608634173609
0.2814255625954827

Ridge
144.10625124944204
0.32711912223395934

Ridge
124.721211176444
0.2725811507614846

Ridge
188.52418288921402
0.044456889094077856

Ridge
153.80427809983902
0.16840725915537702

Ridge
222.77267051852243
0.0016176952888589113

Ridge
281.11937799934606
-0.19468817640505587

Ridge
409.4315103769238
-0.1573841737816517

Ridge
343.4181576537911
0.015647348267101124

Ridge
370.30356610008596
0.029420646282500

Lasso
-------------------